# Space Setup

In [1]:
import sys
import os
import logging
import pandas as pd
import datasets
from datasets import disable_caching; disable_caching()
from pprint import pprint
KEY = '2-OhioT1DM'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_AIDATA': f'_Data/3-Data_AIDATA',
    'DATA_EXTERNAL': f'code/external',
    'DATA_HFDATA': f'_Data/5-Data_HFData',
    'CODE_FN': f'code/pipeline',
    'MODEL_ROOT': f'./_Model',
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'
print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

# os.environ["CUDA_VISIBLE_DEVICES"]="1"

/home/jluo41/CGMLSM-Project/
code/pipeline


/home/jluo41/miniconda3/envs/nix/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Results

In [2]:
path = '_Model/glucopred_nix_optuna'

names = os.listdir(path)


li = []
for name in names:
    try:    
        df_eval = pd.read_parquet(os.path.join(path, name, 'eval.parquet'))
        df_eval['model_name'] = name
        li.append(df_eval)
    except:
        pass

df = pd.concat(li).reset_index(drop=True)
print(df.columns)

columns = ['model_name', 'setname', 
           '000-030min_rMSE',
           '000-060min_rMSE', 
           '000-120min_rMSE',]

df_report = df[columns].sort_values(by='000-120min_rMSE')
df_report

# OhioT1DM	9.358 (0.684)	15.64 (1.078)	26.296 (1.664)

Index(['setname', '000-030min_rMSE', '000-060min_rMSE', '000-120min_rMSE',
       '000-030min_rMSE:all', '000-060min_rMSE:all', '000-120min_rMSE:all',
       'model_name'],
      dtype='object')


,model_name,setname,000-030min_rMSE,000-060min_rMSE,000-120min_rMSE
1,CGMOnlyLSM,[test-id],9.024059,15.894782,26.876165
5,neural_mlp_i289o24_trials10,[test-id],14.972304,20.679605,28.683272
3,neural_transformer_i289o24_trials10,[test-id],27.886044,30.868808,36.652923
4,neural_lstm_i289o24_trials10,[test-id],36.022116,37.169844,38.703427
2,neural_rnn_i289o24_trials10,[test-id],36.102312,37.344436,38.951741
7,neural_gru_i289o24_trials10,[test-id],37.555258,38.572921,40.108142
6,neural_informer_i289o24_trials10,[test-id],35.196747,36.962192,40.203870
0,neural_autoformer_i289o24_trials10,[test-id],36.080422,38.351716,41.394727


In [3]:
path = '_Model/glucopred_nix_optuna'

names = os.listdir(path)


li = []
for name in names:
    try:    
        df_predictions = pd.read_parquet(os.path.join(path, name, 'df_predictions.parquet'))
        df_predictions['model_name'] = name
        li.append(df_predictions)
    except:
        pass

df_predictions_all = pd.concat(li).reset_index(drop=True)
print(df_predictions_all.columns)

# OhioT1DM	9.358 (0.684)	15.64 (1.078)	26.296 (1.664)

Index(['unique_id', 'Inputs', 'Real', 'Predict', 'prediction_time', 'stratum',
       'Splitname', 'model_name', 'split_timebin', 'prediction_hour',
       'age_interval', 'gender', 'diabetes_type'],
      dtype='object')


In [4]:
from nn.eval.seqeval import SeqEvalForOneEvalSet

OUTPUT_LENGTH = 24

horizon_to_se = {
    '30m': [0, 6],
    '1h': [0, 12],
    '2h': [0, 24],
}
CURRENT_POINT = 289 

metric_list = [
    'rMSE', # y_pred, y_real
    'MAE', 
    
    'MAEWithin5',
    'RegionAccu',

    'TIR_error', # y_pred, y_real
] 

evaluator = SeqEvalForOneEvalSet(
    setname = 'all',
    df_case_eval = df_predictions_all,
    x_hist_seq_name = 'Inputs',
    y_real_seq_name = 'Real',
    y_pred_seq_name = 'Predict',
    metric_list = metric_list,
    horizon_to_se = horizon_to_se,
)

df_case_metric, metric_list = evaluator.get_df_case_metric()

['rMSE', 'MAE', 'MAEWithin5', 'RegionAccu', 'TIR_error']


In [ ]:
df_case = df_case_metric
df_case

In [6]:
import numpy as np
import pandas as pd
from scipy.stats import sem, t

def get_case_inoutsplit(x):
    if x['Train'] == True:
        TVT = '0.Train'
    elif x['Valid'] == True:
        TVT = '1.Internal Test'
    elif x['Test'] == True:
        TVT = '2.Temporal Test'
    else:
        print(x['Train'], x['Valid'], x['Test'])
        pass 

    if x['In'] == True:
        InOut = 'In'
    elif x['Out'] == True:
        InOut = 'Out'
    else:
        pass
    split = InOut + '-' + TVT
    if InOut == 'Out':
        split = '3.Hold-Out Test'
    elif InOut == 'In':
        split = TVT
    return split


def map_age_to_group_v1(x):
    if x < 18:
        return 'O: 18'
    elif 18 <= x < 40:
        return 'A: 18-39'
    elif 40 <= x < 65:
        return 'B: 40-64'
    # elif 65 <= x < 80:
    #     return 'C: 65-79'
    # elif x >= 80:
    #     return 'D: 80+'
    elif x >= 65:
        return 'C: 65+'


def map_TIR_to_region(x):
    if 0 < x <= 0.2:
        return '5: OutRange-VeryHigh'
    elif 0.2 < x <= 0.4:
        return '4: OutRange-High'
    elif 0.4 < x <= 0.6:
        return '3: OutRange-Medium'
    elif 0.6 < x <= 0.8:
        return '2: OutRange-Low'
    elif 0.8 < x <= 1:
        return '1: OutRange-VeryLow'
    


def evaluate_df_with_metric(df, metric_name):
    # df should have a name
    metric_values = df[metric_name]
    d = {}
    d['metric_name'] = metric_name
    d['mean'] = metric_values.mean()
    d['sem'] = sem(metric_values)
    d['ci'] = d['sem'] * t.ppf((1 + 0.95) / 2, len(metric_values) - 1)

    # Number of bootstrap samples
    n_bootstrap = 10000
    bootstrap_means = np.empty(n_bootstrap)
    # Perform bootstrap sampling
    for i in range(n_bootstrap):
        # Resample with replacement
        bootstrap_sample = np.random.choice(metric_values, size=len(metric_values), replace=True)
        # Calculate the mean of the bootstrap sample
        bootstrap_means[i] = np.mean(bootstrap_sample)

    # Calculate the bootstrap confidence interval
    confidence_level = 0.95
    lower_percentile = (1 - confidence_level) / 2 * 100
    upper_percentile = (1 + confidence_level) / 2 * 100
    confidence_interval = np.percentile(bootstrap_means, [lower_percentile, upper_percentile])
    # Calculate the bootstrap mean RMSE
    mean_rmse_bootstrap = np.mean(bootstrap_means)

    d['ci_bootstrap'] = confidence_interval
    d['mean_bootstrap'] = mean_rmse_bootstrap
    return d



metric_columns = [
        # 'rMSE_30Min', 'rMSE_1stH', 'rMSE_2ndH', 'rMSE_2H', 
        # 'MAE_30Min', 'MAE_1stH', 'MAE_2ndH', 'MAE_2H',
        # 'MAEWithin5_30Min', 'MAEWithin5_1stH', 'MAEWithin5_2ndH', 'MAEWithin5_2H', 
        # 'MAEWithin10_30Min', 'MAEWithin10_1stH', 'MAEWithin10_2ndH', 'MAEWithin10_2H', 
        # 'RegionAccu_30Min', 'RegionAccu_1stH', 'RegionAccu_2ndH', 'RegionAccu_2H',

       '30m_rMSE', '30m_MAE', '30m_TIR_error',
       '1h_rMSE', '1h_MAE', '1h_TIR_error', 
       '2h_rMSE', '2h_MAE', '2h_TIR_error'

    ]

def process_report_allmetrics(df, metric_columns):
    df = df.copy()
    L = []
    for metric_column in metric_columns:
        d = evaluate_df_with_metric(df, metric_column)
        L.append(d)

    df_report = pd.DataFrame(L) 
    return df_report


def process_report(df, metric_columns = metric_columns):
    df = df.copy()
    columns_y = ['30m', '1h', '2h']
    columns_x = ['rMSE', 'MAE',  'TIR_error']

    # L = []
    # for metric_column in metric_columns:
    #     d = evaluate_df_with_metric(df, metric_column)
    #     L.append(d)
    # df_report = pd.DataFrame(L) 

    # Calculate mean, SEM, and counts
    df_mean = df[metric_columns].mean().reset_index()
    df_sem = df[metric_columns].sem().reset_index()
    df_count = df[metric_columns].count().reset_index()



    # Calculate 90% CI
    confidence_level = 0.95
    df_ci = df_sem.copy()
    for i in range(len(df_ci)):
        n = df_count.iloc[i, 1]
        critical_t = t.ppf((1 + confidence_level) / 2, n - 1)
        df_ci.iloc[i, 1] = df_sem.iloc[i, 1] * critical_t

    # Organize data
    d = {'mean': df_mean, 'sem': df_sem, 'ci': df_ci}
    results = {}
    # results['size'] = len(df)

    # print('the size of the dataset is:', len(df), 'human', df['PID'].nunique())
    for name, df_r in d.items():
        df_r.rename(columns={0: 'Score', 'index': 'FullMetric'}, inplace=True)
        df_r['horizon'] = df_r['FullMetric'].apply(lambda x: x.split('_')[0].strip())
        df_r['Metric']  = df_r['FullMetric'].apply(lambda x: '_'.join(x.split('_')[1:]).strip())
        df_r = df_r.pivot(index='horizon', columns='Metric', values='Score')
        df_r = df_r[columns_x].T
        df_r = df_r[columns_y]
        results[name] = df_r
    return results


def process_performance_output(df, 
                               subcolumns_list, 
                               focus_on_list, 
                               metric, 
                               horizon):
    L_mean = []
    L_sem  = []
    L_ci   = []
    for subcolumns in subcolumns_list:
        for subname, df_sub in df.groupby(subcolumns):
            name = ['.'.join([str(s) for s in i]) for i in list(zip(subcolumns, subname))]
            name = '_'.join(name)
            if len(focus_on_list) > 0:
                indicator = np.mean([focus_on in name for focus_on in focus_on_list])
                if indicator != 1: continue

            print('current name is :', name)
            results = process_report(df_sub)

            df_r = results['mean']
            v = df_r.loc[metric, horizon]
            metric_name = '-'.join([metric, horizon])
            d = {'name': name, metric_name: v}
            for k, v in list(zip(subcolumns, subname)): d[k] = v
            L_mean.append(d)

            df_r = results['sem']
            v = df_r.loc[metric, horizon]
            metric_name = '-'.join([metric, horizon])
            d = {'name': name, metric_name: v}
            for k, v in list(zip(subcolumns, subname)): d[k] = v
            L_sem.append(d)


            df_r = results['ci']
            v = df_r.loc[metric, horizon]
            metric_name = '-'.join([metric, horizon])
            d = {'name': name, metric_name: v}
            for k, v in list(zip(subcolumns, subname)): d[k] = v
            L_ci.append(d)
            
    df_mean = pd.DataFrame(L_mean)
    df_sem = pd.DataFrame(L_sem)
    df_ci = pd.DataFrame(L_ci)

    report_results = {'mean': df_mean, 'sem': df_sem, 'ci': df_ci}
    return report_results


In [ ]:
###############
metric = 'rMSE'
###############

subcolumns_list = [
    ['model_name']
]
focus_on_list = []
horizon_list = ['30m', '1h', '2h']


final_results = {'mean': [], 'sem': [], 'ci': []}
for horizon in horizon_list:
    results = process_performance_output(df_case, subcolumns_list, focus_on_list, metric, horizon)
    value_name = f'{metric}-{horizon}'
    subcolumns = subcolumns_list[0]
    index = subcolumns[0]
    columns = subcolumns[1:]
    for name, dfx in results.items():
        dfx = dfx.rename(columns = {value_name: 'Value'})
        dfx['ValueName'] = value_name
        final_results[name].append(dfx)

df_mean = pd.concat(final_results['mean'])
df_sem  = pd.concat(final_results['sem'])
df_ci  = pd.concat(final_results['ci'])



In [ ]:
df_mean

In [ ]:

Topic = metric

d = {'mean': df_mean, 'ci': df_ci}
results = {}
for name, dfx in d.items():
    dfx = dfx.pivot(index = ['model_name'], columns = 'ValueName', values = 'Value')
    print(dfx.columns)
    custom_order = [f'{metric}-{i}' for i in ['30m', '1h', '2h']]

    dfx = dfx.reindex(columns=custom_order)
    # dfx.to_pickle(path)
    dfx.loc[:, custom_order] = dfx.loc[:, custom_order].round(3)
    results[name] = dfx

    # new_path = path.replace('.p', f'_{name}.p')
    # dfx.to_pickle(new_path)
    # print(new_path)
    display(dfx)


# name = 'ci' # 'mean'
# results[name].to_clipboard()

df_mean = results['mean']
df_ci = results['ci']

df_full = df_mean.copy()
for column in custom_order:
    df_full[column] = df_mean[column].astype(str) + " (" + df_ci[column].astype(str) + ")"

# df_full.to_clipboard()  
df_full